In [1]:
import pandas as pd
import numpy as np
#df_org=pd.read_csv(filepath+"application_train_merge_clear_nomissing .csv")
df_org=pd.read_csv("application_train_merge_clear_nomissing_with_ylabel_dy.csv")

In [3]:
df=df_org

## 資料切分

In [4]:
y_=[]#找出 違約人的索引
for i in (df[df["simple_labelx"]!=0].index):
  y_.append(i)
y_1=[] #找出沒違約人的索引
for i in (df[df["simple_labelx"]==0].index):
  y_1.append(i)
df_split=df.loc[y_]
df_split0=df.loc[y_1]
df_tar1=df_split.reset_index()
df_tar0=df_split0.reset_index()
df_tar_e=df_tar0.iloc[:57886,:] #分成同等分
dfe=pd.concat([df_tar_e,df_tar1],ignore_index=True,axis=0)

In [8]:
100000-57886

42114

In [76]:
## RFE randomforest選出來的前十個(從全部特徵裡選出來的)with prev
df_X = dfe[['DAYS_LAST_PHONE_CHANGE', 'PREV_ANNUITY_median', 'PREV_APPLICATION_max',
       'PREV_CREDIT_max', 'PREV_CREDIT_sum', 'DAYS_DECISION_median',
       'CONTRACT_Cash_loans', 'CONTRACT_Consumer_loans', 'PORTFOLIO_Cash',
       'YIELD_high']]
df_y = dfe['simple_labelx']

In [82]:
## RFE dicisionTree選出來的前十個(從全部特徵裡選出來的)with prev
df_X=dfe[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
       'DAYS_LAST_PHONE_CHANGE', 'PREV_ANNUITY_median', 'PREV_CREDIT_max',
       'PREV_CREDIT_sum', 'PREV_CRE/APP_max', 'DAYS_DECISION_median']]
df_y = dfe['simple_labelx']

In [108]:
df_X=dfe[['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH',
'DAYS_LAST_PHONE_CHANGE', 'PREV_ANNUITY_median','PREV_CREDIT_sum','DAYS_DECISION_median','PORTFOLIO_Cash',
'YIELD_high']]
df_y = dfe['simple_labelx']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X , y = dfe.iloc[:,5:-2] , dfe.iloc[:,-2]
#X , y = df_X  , df_y
stdsc = StandardScaler()
X = stdsc.fit_transform(X)
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.1,#超過10萬9,1分比較好 
                     #random_state=0, 
                     stratify=y)

In [105]:
X_train.shape

(276759, 131)

##　LogisticRegression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)#, random_state=1)
# 建立Logistic模型
logisticModel = LogisticRegression()#random_state=1)
# 使用訓練資料訓練模型
logisticModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = logisticModel.predict(X_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
print('訓練集: ',logisticModel.score(X_train,y_train))
print('測試集: ',logisticModel.score(X_test,y_test))

訓練集:  0.6499222603988714
測試集:  0.6389704612195544


In [11]:
print('分類的預測結果：')
pred = logisticModel.predict(X_test) 
print(pred) #觀察預測結果AA
print('真實數據：')
print(y_test.values)  #觀察真實數據(Test data)

分類的預測結果：
[0. 0. 0. ... 0. 1. 1.]
真實數據：
[0. 0. 1. ... 1. 1. 1.]


In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.6389704612195544

In [18]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)

array([[4567, 1222],
       [1846, 3943]])

## another one

In [106]:
from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import RepeatedStratifiedKFold
#cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)#, random_state=1)
# 建立Logistic模型
logisticModel = LogisticRegression(penalty='l1',C=1.0, solver='liblinear')#random_state=1)
# 使用訓練資料訓練模型
logisticModel.fit(X_train, y_train)
# 使用訓練資料預測分類
predicted = logisticModel.predict(X_train)

In [107]:
print('訓練集: ',logisticModel.score(X_train,y_train))
print('測試集: ',logisticModel.score(X_test,y_test))

訓練集:  0.8101199960976879
測試集:  0.8100936524453695


In [22]:
logisticModel.intercept_

array([0.00051962])

In [23]:
np.set_printoptions(8)

In [24]:
logisticModel.coef_[logisticModel.coef_!=0].shape

(77,)

In [25]:
logisticModel.coef_

array([[ 5.04348193e-02, -1.95373564e-02, -6.64311388e-02,
        -1.99471812e-02, -2.31741011e-02, -2.79149531e-03,
        -6.14923815e-04,  1.50356504e-03,  1.26300680e-02,
         4.38843211e-02,  3.59942938e-02,  0.00000000e+00,
        -3.86904372e-03, -2.46125174e-02,  2.91786762e-02,
         1.19518821e-02, -1.36898197e-02,  3.22267155e-02,
         2.24356855e-02,  4.62983568e-02, -2.29705634e-01,
         5.42748617e-03,  3.12115093e-02, -3.25905313e-02,
         2.06778518e-01,  9.48060070e-02, -5.65571995e-05,
         0.00000000e+00, -1.74435932e-02, -6.41065005e-03,
         6.89085402e-03,  7.01022377e-03, -1.40781947e-06,
        -1.14272807e-02, -1.77841803e-02, -4.42658850e-03,
        -1.05174734e-01, -3.45659746e-01, -5.38818523e-02,
        -1.18067288e-01,  5.10230420e-03, -1.02449838e-02,
         5.48891719e-03, -2.60287348e-02,  1.12400661e-03,
        -5.10586499e-04,  2.31011326e-02,  4.82626631e-03,
        -1.51270090e-02, -1.24561740e-02,  3.60049330e-0

In [95]:
df.columns.tolist()

['Unnamed: 0.1',
 'Unnamed: 0',
 'SK_ID_CURR',
 'TARGET',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'NAME_EDUCATION_TYPE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'HOUR_APPR_PROCESS_START_x',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR',
 'NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'CODE_GENDER_F',
 'CODE_GENDER_M',
 'FLAG_OWN_CAR_N',
 'FLAG_OWN_CAR_Y',
 'FLAG_OWN_REALTY_N',
 'FLAG_OWN_REALTY_Y',
 'NAME_TYPE_SUITE_Children',
 'NAME_TYPE_SUITE_Family',
 'NAME_TYPE_SUITE_Spouse,